In [1]:
!pip install requests

In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
# goals
# - line history
# - % bets
# - % money
# NFL side and total

In [2]:
# scraping function
def scrape_for_soup(base_url, url):
    # send GET request
    response = requests.get(base_url+url)

    # status check
    if response.status_code != 200:
        return(f"Error: {response.status_code}")

    # parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [7]:
base_url = 'https://www.sportsbookreview.com'
url = '/betting-odds/nfl-football/'
soup = scrape_for_soup(base_url, url)

# parse links for game spread and spread history
games = soup.find_all('div', id='tbody-nfl')
main_links = set()
line_links = set()
for game in games:
    for a_tag in game.find_all('a', href=True):
        link = a_tag['href']
        if 'scores' in link:
            main_links.add(link)
        if 'line-history' in link:
            line_links.add(link)

In [8]:
main_links

{'/scores/nfl-football/matchup/304948/',
 '/scores/nfl-football/matchup/304951/',
 '/scores/nfl-football/matchup/304954/',
 '/scores/nfl-football/matchup/304962/',
 '/scores/nfl-football/matchup/304964/',
 '/scores/nfl-football/matchup/304966/',
 '/scores/nfl-football/matchup/304969/',
 '/scores/nfl-football/matchup/304973/',
 '/scores/nfl-football/matchup/304977/',
 '/scores/nfl-football/matchup/304982/',
 '/scores/nfl-football/matchup/304985/',
 '/scores/nfl-football/matchup/304993/',
 '/scores/nfl-football/matchup/304995/',
 '/scores/nfl-football/matchup/304997/',
 '/scores/nfl-football/matchup/304998/',
 '/scores/nfl-football/matchup/305011/'}

In [6]:
print(base_url+game_url)

https://www.sportsbookreview.com/scores/nfl-football/matchup/304917/


In [9]:
# iterate through each link to scrape target data
game_url = list(main_links)[0]
# send request
game_soup = scrape_for_soup(base_url, game_url)
# pull number bets on spread and o/u

# spread bets
spread = game_soup.find('div', text=re.compile('Spread'))
home_perc = spread.next_sibling.get_text()[:-1]
away_perc = spread.previous_sibling.get_text()[:-1]

# over/under bets
total_bets = game_soup.find('div', text=re.compile('Total'))
over_perc = total_bets.previous_sibling.get_text()[:-3]
under_perc = total_bets.next_sibling.get_text()[:-3]
# pull total
tot_div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
# find the span that contains the best total for the over and strip the first element (second element is odds if needed)
total = float(tot_div.find('h4', text='Over').find_parent('div').find_next_sibling('div').find('span').contents[0].strip())

# teams and spread
away_div = game_soup.find('div', class_=re.compile('TeamName'))
home_div = game_soup.find('div', class_=re.compile('TeamName')).findNext('div', class_=re.compile('TeamName'))
home_team = re.findall('.+?(?=\+|-|PK)',home_div.get_text())[0]
away_team = re.findall('.+?(?=\+|-|PK)',away_div.get_text())[0]
away_spread = away_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
home_spread = home_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
print(f'The spread is {home_spread}')
print(f'% bets {home_team} {home_spread}: {home_perc}')
print(f'% bets {away_team} {away_spread} % bets: {away_perc} \n')
print(f'The total is {str(total)}')
print(f'% bets over {str(total)}: {over_perc}')
print(f'% bets under {str(total)}:{under_perc}')

The spread is +3
% bets SeattleSeahawks +3: 41
% bets BuffaloBills -3 % bets: 59 

The total is 47.0
% bets over 47.0: 68
% bets under 47.0:32


In [13]:
cols =['home_team','away_team', 'home_spread', 'home_perc', 'away_perc', 'total', 'over_perc', 'under_perc']
df = pd.DataFrame(columns=cols)

# iterate through each link to scrape target data
for game_url in list(main_links):
    # send request
    game_soup = scrape_for_soup(base_url, game_url)
    # pull number bets on spread and o/u

    # spread bets
    spread = game_soup.find('div', text=re.compile('Spread'))
    home_perc = spread.next_sibling.get_text()[:-1]
    away_perc = spread.previous_sibling.get_text()[:-1]

    # over/under bets
    total_bets = game_soup.find('div', text=re.compile('Total'))
    over_perc = total_bets.previous_sibling.get_text()[:-3]
    under_perc = total_bets.next_sibling.get_text()[:-3]
    # pull total
    tot_div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
    # find the span that contains the best total for the over and strip the first element (second element is odds if needed)
    total = float(tot_div.find('h4', text='Over').find_parent('div').find_next_sibling('div').find('span').contents[0].strip())

    # teams and spread
    away_div = game_soup.find('div', class_=re.compile('TeamName'))
    home_div = game_soup.find('div', class_=re.compile('TeamName')).findNext('div', class_=re.compile('TeamName'))
    home_team = re.findall('.+?(?=\+|-|PK)',home_div.get_text())[0]
    away_team = re.findall('.+?(?=\+|-|PK)',away_div.get_text())[0]
    away_spread = away_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
    home_spread = home_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
    
    # add data to df
    row = dict()
    row['home_team'] = home_team
    row['away_team'] = away_team
    row['home_spread'] = home_spread
    row['home_perc'] = home_perc
    row['away_perc'] = away_perc
    row['total'] = total
    row['over_perc'] = over_perc
    row['under_perc'] = under_perc

    df = df.append(row, ignore_index=True)

In [11]:
cols =['home_team','away_team', 'home_spread', 'home_perc', 'away_perc', 'total', 'over_perc', 'under_perc']
df = pd.DataFrame(columns=cols)

In [14]:
df

,home_team,away_team,home_spread,home_perc,away_perc,total,over_perc,under_perc
0,SeattleSeahawks,BuffaloBills,+3,41,59,47.0,68,32
1,DetroitLions,TennesseeTitans,-11.5,75,25,44.5,65,35
2,JacksonvilleJaguars,Green BayPackers,+4,29,71,48.5,51,49
3,Tampa BayBuccaneers,AtlantaFalcons,+2,44,56,45.5,62,38
4,New EnglandPatriots,N.Y. JetsJets,+7,43,57,41.0,48,52
5,CincinnatiBengals,PhiladelphiaEagles,-2.5,56,44,47.5,55,45
6,L.A. ChargersChargers,New OrleansSaints,-7,53,47,40.5,47,53
7,WashingtonCommanders,ChicagoBears,+2,59,41,43.0,71,29
8,ClevelandBrowns,BaltimoreRavens,+9,27,73,44.5,56,44
9,DenverBroncos,CarolinaPanthers,-10,75,25,41.5,34,66


In [9]:
# next: pull total history/movement if possible
line_url = list(line_links)[0]
line_soup = scrape_for_soup(base_url, line_url)

In [10]:
print(base_url+line_url)

https://www.sportsbookreview.com/betting-odds/nfl-football/line-history/304842/


In [10]:
div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
print(div)

<div class="row GameMatchup_mainColumn__cILb0"><aside class="col-12 col-lg-3"><div class="mt-2"><h2 class="d-flex align-items-center justify-content-between mb-0 py-1 px-3 bg-dark h4" data-bs-theme="dark">Betting Consensus</h2><div class="d-flex justify-content-center pt-2"><div class="col-4 d-flex flex-column justify-content-center text-center text-primary fx-bold pb-3 GameMatchup_TeamName__hOY6B"><p class="h4">Buffalo<br/>Bills</p><span class="h5 mb-2 text-secondary opacity-75">-3</span></div><div class="d-flex justify-content-center align-items-center mx-2 mt-4 rounded-circle fs-7 text-white bg-dark text-center GameMatchup_TagCircle__FABto">vs</div><div class="col-4 d-flex flex-column justify-content-center text-center text-primary fx-bold pb-3 GameMatchup_TeamName__hOY6B"><p class="h4">Seattle<br/>Seahawks</p><span class="h5 mb-2 text-secondary opacity-75">+3</span></div></div><div class="d-flex flex-column w-100 border-top undefined Rows_Rows__wF9TM"><div class="d-flex align-conte